In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import base64
import json
from groq import Groq
import time

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def process_image(image_path, client):
    base64_image = encode_image(image_path)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
Analyze the image and extract information into the following JSON structure:
{
    "purpose": "Provide guidance on product specification attributes",
    "critical_attributes": [
        {
            "Attributes": "THREAD SIZE",
            "Tolerances": "IDENTICAL",
            "Tolerance Variability": null,
            "Item Notes": null
        }
    ],
    "notes": {
        "identical": "Strict matching required",
        "upgradable": "Potential upgrade conditions",
        "flexible": "Modifications allowed",
        "nonCritical": "Optional attributes"
    }
}

Ensure:
- Use exact values from the image
- Maintain clean, valid JSON format
- Include all visible attributes
- Use null for empty/undefined fields
- Ensure json variables according to the name in image. like some images have attributes and some have critical attributes.
"""
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
        model="llama-3.2-11b-vision-preview",
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"}
    )

    return json.loads(chat_completion.choices[0].message.content)


# Path to your folder containing images
image_folder = "" #Replace Folder path here

# Path to save JSON files
json_folder = os.path.join(image_folder, "jsonfiles")

# Create the JSON folder if it doesn't exist
os.makedirs(json_folder, exist_ok=True)

# Initialize the Groq client
client = Groq(
    api_key=""  # Replace with your API key
)

# Iterate over all images in the folder
for image_file in os.listdir(image_folder):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        image_path = os.path.join(image_folder, image_file)
        while True:  # Retry until successful
            try:
                # Process each image and get JSON
                result_json = process_image(image_path, client)

                # Save JSON in the jsonfiles folder with the image's name
                json_output_path = os.path.join(json_folder, f"{os.path.splitext(image_file)[0]}.json")
                with open(json_output_path, "w") as json_file:
                    json.dump(result_json, json_file, indent=4)
                print(f"Processed and saved: {image_file} as {json_output_path}")
                break  # Exit retry loop on success
            except Exception as e:
                print(f"Error processing {image_file}: {e}. Retrying...")
                time.sleep(2)  # Add a short delay before retrying

Processed and saved: Anchors- Conical Plug Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors- Conical Plug Anchors.json
Processed and saved: Anchors- Drop-In Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors- Drop-In Anchors.json
Processed and saved: Anchors- Female-Threaded Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors- Female-Threaded Anchors.json
Processed and saved: Anchors- Screw Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors- Screw Anchors.json
Processed and saved: Anchors- Sleeve Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors- Sleeve Anchors.json
Processed and saved: Anchors-Wedge Anchors.png as /content/drive/MyDrive/Screenshots/jsonfiles/Anchors-Wedge Anchors.json
Processed and saved: Blind Rivets.png as /content/drive/MyDrive/Screenshots/jsonfiles/Blind Rivets.json
Processed and saved: Cotter Pins.png as /content/drive/MyDrive/Screenshots/jsonfiles/Cotter Pins.json
Processed an